In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [ ]:
train_dir = '/content/drive/MyDrive/Distillation/newdataset/train'
test_dir = '/content/drive/MyDrive/Distillation/newdataset/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5025 images belonging to 13 classes.
Found 1256 images belonging to 13 classes.


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [ ]:
model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=[checkpoint, early_stopping])

Epoch 1/80
158/158 [==============================] - 4242s 27s/step - loss: 0.9949 - accuracy: 0.7099 - val_loss: 0.4590 - val_accuracy: 0.8877
Epoch 2/80
158/158 [==============================] - 28s 179ms/step - loss: 0.3246 - accuracy: 0.9204 - val_loss: 0.2814 - val_accuracy: 0.9188
Epoch 3/80
158/158 [==============================] - 28s 180ms/step - loss: 0.1968 - accuracy: 0.9516 - val_loss: 0.1705 - val_accuracy: 0.9530
Epoch 4/80
158/158 [==============================] - 29s 181ms/step - loss: 0.1453 - accuracy: 0.9666 - val_loss: 0.1753 - val_accuracy: 0.9514
Epoch 5/80
158/158 [==============================] - 33s 209ms/step - loss: 0.1134 - accuracy: 0.9721 - val_loss: 0.1065 - val_accuracy: 0.9745
Epoch 6/80
158/158 [==============================] - 29s 182ms/step - loss: 0.0844 - accuracy: 0.9785 - val_loss: 0.1174 - val_accuracy: 0.9650
Epoch 7/80
158/158 [==============================] - 29s 180ms/step - loss: 0.0727 - accuracy: 0.9847 - val_loss: 0.0911 - val_ac

In [ ]:
!mv /content/best_model.keras /content/drive/MyDrive/Distillation/

In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/Distillation/best_model.keras')

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

40/40 [==============================] - 6s 148ms/step - loss: 0.0508 - accuracy: 0.9857

Test Loss: 0.0508
Test Accuracy: 98.57%
40/40 [==============================] - 5s 126ms/step
Accuracy: 98.56687898089172
Precision: 98.59139865046481
Recall: 98.56687898089172
F1 Score: 98.56306353788125

Classification Report:
              precision    recall  f1-score   support

 cong_chieng     1.0000    0.9833    0.9916        60
     dan_bau     0.9875    1.0000    0.9937        79
      dan_co     0.9783    0.9574    0.9677        47
      dan_da     1.0000    0.9837    0.9918       123
     dan_day     0.9787    1.0000    0.9892        92
  dan_nguyet     0.9919    0.9840    0.9880       125
     dan_sen     0.9701    0.9924    0.9811       131
  dan_t_rung     0.9941    0.9883    0.9912       171
    dan_tinh     0.9919    0.9919    0.9919       124
   dan_tranh     0.9702    0.9939    0.9819       164
   dan_ty_ba     0.9600    1.0000    0.9796        48
        khen     1.0000    0.92